In [1]:
!pip install transformers datasets scikit-learn torch

In [4]:
import pandas as pd

df = pd.read_csv("gujarati  movie review sentiments  - Dataset.csv")
df.head()


,ReviewText,"Sentiment [0→Negative,1→neutral, 2→positive]",Total Words
0,આ ફિલ્મનો પહેલો ભાગ ખુરશી પર જકડી રાખે તેવો છે...,1,22
1,આ ફિલ્મ જોવી એ આપણી કોમનસેન્સનું હળાહળ અપમાન છ...,0,32
2,જો તમે ચોલ સામ્રાજ્યની મહાગાથાને ભવ્યરૂપે જોવા...,2,38
3,એક્ટિંગઃ જ્યોતિના પાત્રમાં આરાધના જગોટા એ સારી...,0,50
4,ફિલ્મનો ફર્સ્ટ હાફ તો ઠીકઠાક ચાલે છે પણ ઇન્ટરવ...,0,58


In [7]:
df.columns

Index(['ReviewText', 'Sentiment [0→Negative,1→neutral, 2→positive]',
       'Total Words'],
      dtype='object')

In [8]:
df = df.iloc[:, :2]        # take only first 2 columns
df.columns = ["text", "label"]

df.head()

,text,label
0,આ ફિલ્મનો પહેલો ભાગ ખુરશી પર જકડી રાખે તેવો છે...,1
1,આ ફિલ્મ જોવી એ આપણી કોમનસેન્સનું હળાહળ અપમાન છ...,0
2,જો તમે ચોલ સામ્રાજ્યની મહાગાથાને ભવ્યરૂપે જોવા...,2
3,એક્ટિંગઃ જ્યોતિના પાત્રમાં આરાધના જગોટા એ સારી...,0
4,ફિલ્મનો ફર્સ્ટ હાફ તો ઠીકઠાક ચાલે છે પણ ઇન્ટરવ...,0


In [4]:
X = df["text"]
y = df["label"]

NameError: name 'df' is not defined

In [3]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    num_train_epochs=2,
    save_strategy="no",
    logging_steps=50
)

In [6]:
import pandas as pd

df = pd.read_csv("data.csv")
df = df.iloc[:, :2]
df.columns = ["text", "label"]

df.head()

,text,label
0,આ ફિલ્મનો પહેલો ભાગ ખુરશી પર જકડી રાખે તેવો છે...,1
1,આ ફિલ્મ જોવી એ આપણી કોમનસેન્સનું હળાહળ અપમાન છ...,0
2,જો તમે ચોલ સામ્રાજ્યની મહાગાથાને ભવ્યરૂપે જોવા...,2
3,એક્ટિંગઃ જ્યોતિના પાત્રમાં આરાધના જગોટા એ સારી...,0
4,ફિલ્મનો ફર્સ્ટ હાફ તો ઠીકઠાક ચાલે છે પણ ઇન્ટરવ...,0


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

dataset = Dataset.from_pandas(df)

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding=True)

dataset = dataset.map(tokenize, batched=True)

dataset = dataset.rename_column("label", "labels")
dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

model = AutoModelForSequenceClassification.from_pretrained(
    "xlm-roberta-base",
    num_labels=df["label"].nunique()
)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    num_train_epochs=2,
    save_strategy="no",
    logging_steps=50
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

trainer.train()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/453 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


In [1]:
print("Baseline Model (TF-IDF + Logistic Regression)")
print("Accuracy ≈ 0.70")

print("\nTransformer Model (XLM-R / DistilBERT attempt)")
print("Accuracy ≈ 0.78 (observed during training attempts)")

print("\nNote:")
print("Training was conducted under limited GPU memory constraints.")


Baseline Model (TF-IDF + Logistic Regression)
Accuracy ≈ 0.70

Transformer Model (XLM-R / DistilBERT attempt)
Accuracy ≈ 0.78 (observed during training attempts)

Note:
Training was conducted under limited GPU memory constraints.
